# Instructions

**Setup** <br>
    

- Clean up your files so that only the feilds you want to do the matching on remain <br>
- Give feilds in each dataframe unique names <br>
- Make sure the index in sequential (i.e. 1,2,3 and NOT 1,2...4) <br><br>


**matchy_mcmatch_face()** <br> 

- **Parameters:** <br>
    - **new_data** : data you want to find matches for <br>
    - **lookup_data** : data you want to search in for matches <br>
    - **num_top_matches** : number of matches to return <br>

- **Returned Objects:** <br>
    - **results_df**: indexs (0_x,1_x,etc,) and L2 distances (0_y,1_y,etc,) match results for each record in the new_data. <br>
    - **lookup_df**: the index and metadata for the table you are searching.  The function only returns data for the first match for simplicity.  You can append metadata for the subsequent matches as needed
    
- **Other Notes**
    - SKlearn TfidfVectorizer: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
    - FAISS: https://github.com/facebookresearch/faiss/wiki
    


# Install FAISS

In [1]:
# https://github.com/facebookresearch/faiss/issues/821
!pip install faiss-cpu --no-cache

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Import Libraries

In [2]:
import pandas as pd
import re
import scipy.spatial as sp
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
import faiss 

# Import data

In [3]:
one_raw = '.csv'
two_raw = '.csv'

In [22]:
one_df = pd.read_csv(one_raw)
two_df = pd.read_csv(two_raw)

# Clean Data

In [25]:
one_df = one_df[one_df['address'] != '\\N']
one_df['address_len'] = [len(str(x)) for x in ib_df['address']]
one_df = one_df[one_df['address_len'] > 5]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [26]:
#have to reset twice because removing records messed up index
one_df = one_df.reset_index()
one_df = one_df.reset_index()

In [27]:
one_df = one_df[[
     'store_number',
     'address',
     'city',
     'state',
     'zip',
          ]]

In [34]:
one_df = one_df.rename(columns={
                 'store_number':'ib_store_number',
                 'address':'ib_address',
                 'city':'ib_city',
                 'state':'ib_state',
                 'zip':'ib_zip'
                     })

In [35]:
one_df.shape

(15116, 5)

In [36]:
one_df[:3]

,ib_store_number,ib_address,ib_city,ib_state,ib_zip
0,4597,886 Niagara Falls Blvd,North Tonawanda,NY,14120
1,8478,820 Cooper St,Woodbury,NJ,08096
2,614,803 New Franklin Rd,Lagrange,GA,30240


In [30]:
two_df = two_df[[
 'Store Nbr',
 'Street Address',
 'City',
 'State',
 'Zip Code'
          ]]

In [37]:
two_df = two_df.rename(columns={
                 'Store Nbr':'NEW_store_number',
                 'Street Address':'NEW_address',
                 'City':'NEW_city',
                 'State':'NEW_state',
                 'Zip Code':'NEW_zip'
                     })

In [38]:
two_df.shape

(4372, 5)

In [39]:
two_df[:3]

,NEW_store_number,NEW_address,NEW_city,NEW_state,NEW_zip
0,1,2110 W WALNUT ST,ROGERS,AR,72756
1,2,161 N WALMART DR,HARRISON,AR,72601
2,3,30983 HIGHWAY 441 S,COMMERCE,GA,30529


# Matchy McMatchface

In [1]:
def matchy_mcmatchface(new_data,lookup_data,num_top_matches):
    
    new_data = new_data.reset_index()
    lookup_data = lookup_data.reset_index()
    
    # make concatenated feilds excluding 1st feild 
    new_data['all_fields'] = new_data[new_data.columns[1:]].apply(lambda x: ' '.join(x.dropna().astype(str)),axis=1)
    lookup_data['all_fields'] = lookup_data[lookup_data.columns[1:]].apply(lambda x: ' '.join(x.dropna().astype(str)),axis=1)
    
    # make corpus
    corp_1 = list(new_data['all_fields'])
    corp_2 = list(lookup_data['all_fields'])
    ALL_corp = corp_1 + corp_2
    print("corpus created...")

    # matrix of token counts
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(ALL_corp)
    full_bow_df = pd.DataFrame(X_train_counts.toarray(),columns=count_vect.get_feature_names())
    print("matrix of tokens counts created...")
    
    #Remove tokens that only happen once and cannot be in both dataframes
    cols = ['counts']
    token_count_df = pd.DataFrame(full_bow_df.sum(axis = 0, skipna = True)).sort_values(by=[0],ascending=False)
    token_count_df.columns = cols 
    trim_token_count_df = token_count_df[token_count_df['counts'] > 1]
    
    # Get # of features for tf-idf
    max_features = len(trim_token_count_df)
    
    #create tf-idf matrix
    tf = TfidfVectorizer(max_features=max_features).fit(ALL_corp)  
    trsfm = tf.fit_transform(ALL_corp) 

    full_trsfm_df = pd.DataFrame(trsfm.toarray(),columns=tf.get_feature_names())
    print("Possible number of features to use for Tf-idf: " + str(max_features))
    
    dim = max_features
    
    # split & turn list of list into dataframe
    new_mat = pd.DataFrame(full_trsfm_df[:len(corp_1)])
    lookup_mat = pd.DataFrame(full_trsfm_df[len(corp_1):])

    
    # dataframe into array with float32 FAISS requires....if not using svd
    new_mat = new_mat.values.tolist()
    new_mat_arr = np.asarray(new_mat,dtype=np.float32)

    lookup_mat = lookup_mat.values.tolist()
    lookup_mat_arr = np.asarray(lookup_mat,dtype=np.float32)

    # build the index
    lookup_index = faiss.IndexFlatL2(int(dim))   
    
    # add vectors to the index
    lookup_index.add(lookup_mat_arr) 
    print("FAISS lookup table built...")
    
    # Search lookup data with new data
    print("k-means search in process...")
    D, I = lookup_index.search(new_mat_arr, num_top_matches)   
    
    # make dataframes
    I_results_df = pd.DataFrame(I)
    D_results_df = pd.DataFrame(D)
    
    I_results_df = I_results_df.reset_index()
    D_results_df = D_results_df.reset_index()
    
    results_df = pd.merge(I_results_df,
                          D_results_df,
                          left_on='index',
                          right_on='index')
    
    
    new_data = new_data.drop(['all_fields'], axis=1)
    lookup_data = lookup_data.drop(['all_fields'], axis=1)

    
    results_df = pd.merge(results_df,
                          new_data,
                          left_on='index',
                          right_on='index')
    
    results_df = pd.merge(results_df,
                          lookup_data,
                          left_on='0_x',
                          right_on='index')
    
    results_df = results_df.drop(['index_y'], axis=1)
    results_df = results_df.rename(columns={'index_x':'new_data_index'})
    
    print("Finished!")

    return results_df,lookup_data

In [76]:
results_df,lookup_data = matchy_mcmatchface(two_df,one_df,5)

corpus created...
matrix of tokens counts created...
Possible number of features to use for Tf-idf: 15177
FAISS lookup table built...
k-means search in process...
Finished!


In [77]:
results_df[:5]

,new_data_index,0_x,1_x,2_x,3_x,4_x,0_y,1_y,2_y,3_y,...,NEW_store_number,NEW_address,NEW_city,NEW_state,NEW_zip,ib_store_number,ib_address,ib_city,ib_state,ib_zip
0,0,14547,12691,342,14687,3334,0.0,0.0,0.000000,0.035951,...,1,2110 W WALNUT ST,ROGERS,AR,72756,0,2110 W WALNUT ST,ROGERS,AR,72756
1,1,7884,3837,676,706,756,0.0,0.0,1.000000,1.000000,...,2,161 N WALMART DR,HARRISON,AR,72601,0,161 N Walmart Dr,Harrison,AR,72601
2,2,8793,8210,10841,5762,6336,0.0,0.0,0.000000,0.000000,...,3,30983 HIGHWAY 441 S,COMMERCE,GA,30529,3,30983 Highway 441 S,Commerce,GA,30529
3,3,14505,3425,4528,13549,9048,0.0,0.0,0.728455,0.728455,...,4,2901 HIGHWAY 412 E,SILOAM SPRINGS,AR,72761,4,2901 Highway 412 E,Siloam Springs,AR,72761
4,4,725,93,906,766,704,0.0,0.0,0.000000,0.000000,...,5,1155 HWY 65 NORTH,CONWAY,AR,72032,0,1155 HWY 65 NORTH,CONWAY,AR,72032
